# wellpathpy prototype

## Goal

To build a light package to load well deviations

## Objectives

1. load well deviation in one of <_n_> formats:
    * meta data (header, rkb, dfe, rt)
    * md, incl, azi
    * mE, mN, depth
    * other ?,?,?
2. interpolate survey using one of these methods:
    * minimum curvature method
    * radius of curvature method
    * tangential method
    * other ?
3. calculate dog-leg severity
4. calculate depth references using header data if available: MD, TVD, TVDSS
5. return interpolated deviation in all <_n_> input formats and in all depth references if possible

## Sources of equations:

- [petrowiki](https://petrowiki.org/Calculation_methods_for_directional_survey)
- [Crain's Petrophysical Handbook](https://www.spec2000.net/19-dip13.htm)
- [drillingformulas](http://www.drillingformulas.com)

## software _architecture_

(get input from Jørgen: `jokva`)

- code to `list` (will then also accept `nd.array`)
- data reader (chooses files to accept: `*.csv`, `*.xlsx`)
- guess data types if not provided by user?
- library of interpolation functions
- depth calculation function
- deploy to [pypi](https://pypi.org/)

In [1]:
import numpy as np
import pandas as pd

In [18]:
df = pd.read_csv('./data/Well_Surveys_Projected_to_TD.csv', skiprows=4, usecols=['MD[m]','Inc[deg]','Azi[deg]'])
df.head()

,MD[m],Inc[deg],Azi[deg]
0,0,0.0,0.00
1,76.29,0.9,7.19
2,104.36,0.9,17.20
3,134,0.9,15.59
4,162.73,1.1,6.39


In [19]:
df.drop(df.tail(1).index,inplace=True)
df.dropna(inplace=True)
df.to_csv('./data/deviation.csv', index=False)

In [31]:
def read_deviation(fname):
    """
    Read a deviation file into memory for processing
    
    Args:
        filename
        
    Return:
        pd.DataFrame
        
    """
    return pd.read_csv(fname)


In [32]:
read_deviation('./data/deviation.csv').head()

,MD[m],Inc[deg],Azi[deg]
0,0.00,0.0,0.00
1,76.29,0.9,7.19
2,104.36,0.9,17.20
3,134.00,0.9,15.59
4,162.73,1.1,6.39


In [37]:
# should a deviation be a Class?
def get_headers(datum='kb', units='m', elevation=0.):
    """
    Record deviation header information needed for depth reference calculation into DataFrame.
    
    Parameters
    ----------
    datum : str, default 'kb', {'kb', 'dfe', 'rt'}
            'kb' (kellybushing), 'dfe' (drill floor elevation), 'rt' (rotary table)
    units: str, default 'm', {'m', 'ft'}
            'm' (metres), 'ft' (feet)
    elevation: float, default 0., 
            <datum> <elevation> in <units> above mean sea level
    
    Returns
    -------
    DataFrame
        deviation header dictionnary
    """
    dev_header = {}
    dev_header.update({'datum': datum, 'units': units, 'elevation': elevation})
    
    return dev_header

In [38]:
get_headers(datum='kb', units='m', elevation=12)

{'datum': 'kb', 'units': 'm', 'elevation': 12}

In [40]:
def mdtotvd(deviation, method='tan'):
    """
    Calculate TVD from given method.
    
    Parameters
    ----------
    deviation: DataFrame in MD, incl, azi
    method: str, default 'tangential', {'tan', 'avtan', 'baltan', 'merc', 'radcurv', 'mincurv'}
            'tan' (tangential method),  'avtan' (average tangential method),
            'baltan' (balanced tangential method), 'merc' (mercury method),
            'radcurv' (radius of curvature method), 'mincurv' (minimum curvature method)
            method definitions: [Crain's Petrophysical Handbook](https://www.spec2000.net/19-dip13.htm)
            
    Returns
    -------
    DataFrame
        deviation survey converted to TVD, easting, northing
        
    To Do
    -----
    implement methods
            
    """
    
    return deviation

In [ ]:
mdtotvd()